## Загрузим нужные библиотеки

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np


%matplotlib inline

In [ ]:
df = pd.read_csv("train_dataset_train (1).csv")

#df = df.sample(10000)
print(df.shape)
df.head()

In [ ]:
df = pd.read_csv("train_dataset_train (1).csv")

mapper_s_el = df.groupby(['station_id']).agg({'entrance_id':lambda x: Counter(x).most_common()[0][0],
                               'line_id':lambda x: Counter(x).most_common()[0][0],}).reset_index()

df['back_time'] = pd.to_timedelta(df['time_to_under'],unit='m') + pd.to_datetime(df['pass_dttm'])
joined = df.copy()
joined['pass_dttm'] = pd.to_timedelta(joined['time_to_under'],unit='m') + pd.to_datetime(joined['pass_dttm'])
joined['id'] = df['id'].max()+10 +joined['id']

joined['temp'] = joined['station_id'].copy()
joined['station_id'] = joined['label'].copy()
joined['label'] = joined['temp'].copy()
joined.drop(['temp'], axis=1, inplace=True)
#joined['label'], joined['station_id'] = joined['station_id'], joined['label']
joined['time_to_under'] = None
joined.drop(['entrance_id','line_id'],axis=1, inplace=True)
joined = joined.merge(mapper_s_el)

#df = df.sample(10000)
print(joined.shape)

df = pd.concat([df, joined])
df = df.sort_values(by='ticket_id')
print(df.shape)
df.head(10)

In [ ]:
features = ['ticket_type_nm','station_id','line_id','pass_hour','pass_weekday','pass_minute',
           'schoolclidren','student','days_num','only_one','entrance_id']


def handle(df):
    

    df['pass_dttm'] = pd.to_datetime(df['pass_dttm'])
    df['pass_hour'] = df.pass_dttm.dt.hour
    df['pass_weekday'] = df.pass_dttm.dt.weekday
    df['pass_minute'] = df.pass_dttm.dt.minute
    
    df['schoolclidren'] = df.ticket_type_nm.apply(lambda x: int('учащ' in x))
    df['student'] =  df.ticket_type_nm.apply(lambda x: int('студен' in x or 'аспир' in x))
    df['days_num'] = df.ticket_type_nm.apply(
        lambda x: 90 if '90 дней' in x else (30 if '30 д' in x else -1)
    )
    df['only_one'] = df.ticket_type_nm.apply(lambda x: int('един' in x)).astype(np.int32)
    

    cats_features = []

    l = ['ticket_type_nm','line_id','station_id','entrance_id']
    for c in features:
        col_type = df[c].dtype
        if col_type == 'object' or col_type.name == 'category' or c in l:
            df[c] = df[c].astype('category')
            cats_features.append(c)

            
    return df, cats_features

def encode(df, col, item2id=None):
    
    if item2id is None:
    
        item2id = {v:k for k, v in enumerate(df[col].unique())}
        id2item = {v:k for k, v in item2id.items()}
    
    df[col] = df[col].apply(lambda x: item2id.get(x,0)).astype('category')
    print(len(item2id))
    
    return df, item2id



In [ ]:
df, cat_features = handle(df)
df, ticket_encoder = encode(df, 'ticket_type_nm')
df, station_encoder = encode(df, 'station_id')
df, line_encoder = encode(df, 'line_id')
df, entrance_encoder = encode(df, 'entrance_id')
df, label_encoder = encode(df, 'label')
df['label'] = df['label'].astype(np.int32)
cat_features

In [ ]:
train = df#.sample(100_000)
print(train.shape)

In [ ]:
from scipy import optimize
from scipy import special

class FocalLoss:
    """
    source: https://maxhalford.github.io/blog/lightgbm-focal-loss/
    """

    def __init__(self, gamma, alpha=None):
        self.alpha = alpha
        self.gamma = gamma

    def at(self, y):
        if self.alpha is None:
            return np.ones_like(y)
        return np.where(y, self.alpha, 1 - self.alpha)

    def pt(self, y, p):
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return np.where(y, p, 1 - p)

    def __call__(self, y_true, y_pred):
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        return -at * (1 - pt) ** self.gamma * np.log(pt)
    def grad(self, y_true, y_pred):
        y = 2 * y_true - 1  # {0, 1} -> {-1, 1}
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        g = self.gamma
        return at * y * (1 - pt) ** g * (g * pt * np.log(pt) + pt - 1)

    def hess(self, y_true, y_pred):
        y = 2 * y_true - 1  # {0, 1} -> {-1, 1}
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        g = self.gamma

        u = at * y * (1 - pt) ** g
        du = -at * y * g * (1 - pt) ** (g - 1)
        v = g * pt * np.log(pt) + pt - 1
        dv = g * np.log(pt) + g + 1

        return (du * v + u * dv) * y * (pt * (1 - pt))

    def init_score(self, y_true):
        res = optimize.minimize_scalar(
            lambda p: self(y_true, p).sum(),
            bounds=(0, 1),
            method='bounded'
        )
        p = res.x
        log_odds = np.log(p / (1 - p))
        return log_odds
    def lgb_obj(self, preds, train_data):
        y = train_data.get_label()
        p = special.expit(preds)
        return self.grad(y, p), self.hess(y, p)

    def lgb_eval(self, preds, train_data):
        y = train_data.get_label()
        p = special.expit(preds)
        is_higher_better = False
        return 'focal_loss', self(y, p).mean(), is_higher_better

In [ ]:
from joblib import Parallel, delayed
from sklearn.multiclass import _ConstantPredictor
from sklearn.preprocessing import LabelBinarizer
from scipy import special
import lightgbm as lgbm

class OneVsRestLightGBMWithCustomizedLoss:
    """
    source: https://towardsdatascience.com/multi-class-classification-using-focal-loss-and-lightgbm-a6a6dec28872
    """

    def __init__(self, loss, n_jobs=3):
        self.loss = loss
        self.n_jobs = n_jobs

    def fit(self, X, y, **fit_params):
        self.label_binarizer_ = LabelBinarizer(sparse_output=True)
        Y = self.label_binarizer_.fit_transform(y)
        Y = Y.tocsc()
        self.classes_ = self.label_binarizer_.classes_
        columns = (col.toarray().ravel() for col in Y.T)
        if 'eval_set' in fit_params:
            # use eval_set for early stopping
            X_val, y_val = fit_params['eval_set'][0]
            Y_val = self.label_binarizer_.transform(y_val)
            Y_val = Y_val.tocsc()
            columns_val = (col.toarray().ravel() for col in Y_val.T)
            self.results_ = Parallel(n_jobs=self.n_jobs)(delayed(self._fit_binary)
                                                         (X, column, X_val, column_val, **fit_params) for
                                                         i, (column, column_val) in
                                                         enumerate(zip(columns, columns_val)))
        else:
            # eval set not available
            self.results_ = Parallel(n_jobs=self.n_jobs)(delayed(self._fit_binary)
                                                         (X, column, None, None, **fit_params) for i, column
                                                         in enumerate(columns))

        return self
    def _fit_binary(self, X, y, X_val, y_val, **fit_params):
        unique_y = np.unique(y)
        init_score_value = self.loss.init_score(y)
        if len(unique_y) == 1:
            estimator = _ConstantPredictor().fit(X, unique_y)
        else:
            fit = lgbm.Dataset(X, y, init_score=np.full_like(y, init_score_value, dtype=float))
            filtering = ['eval_set', 'early_stopping_rounds', 'verbose_eval', 'num_boost_round']
            local_fit_params = {item:value for item, value
                                in fit_params.items() if item!='eval_set' and item != 'es'}
            
            if 'num_boost_round' in fit_params:
                num_boost_round = fit_params['num_boost_round']
            else:
                num_boost_round = 100
                
            if 'early_stopping_rounds' in fit_params:
                early_stopping_rounds = fit_params['early_stopping_rounds']
            else:
                early_stopping_rounds = 10
                
            if 'verbose_eval'  in fit_params:
                verbose_eval = fit_params['verbose_eval']
            else:
                verbose_eval = 10
                
            if 'eval_set' in fit_params:
                val = lgbm.Dataset(X_val, y_val, init_score=np.full_like(y_val, init_score_value, dtype=float),
                                  reference=fit)
        
                estimator = lgbm.train(params=local_fit_params,
                                       train_set=fit,
                                       valid_sets=(fit, val),
                                       valid_names=('fit', 'val'),
                                       fobj=self.loss.lgb_obj,
                                       feval=self.loss.lgb_eval,
                                       num_boost_round=num_boost_round,
                                       #early_stopping_rounds=early_stopping_rounds,
                                       #verbose_eval=verbose_eval
                                       callbacks = [fit_params['es']]
                                      )
            else:
                                   
                estimator = lgbm.train(params=local_fit_params,
                                       train_set=fit,
                                       fobj=self.loss.lgb_obj,
                                       feval=self.loss.lgb_eval,
                                       num_boost_round=num_boost_round,
                                       early_stopping_rounds=early_stopping_rounds,
                                       #verbose_eval=verbose_eval
                                       callbacks = fit_params['es']
                                      )

        return estimator, init_score_value

    def predict(self, X):

        n_samples = X.shape[0]
        maxima = np.empty(n_samples, dtype=float)
        maxima.fill(-np.inf)
        argmaxima = np.zeros(n_samples, dtype=int)

        for i, (e, init_score) in enumerate(self.results_):
            margins = e.predict(X, raw_score=True)
            prob = special.expit(margins + init_score)
            np.maximum(maxima, prob, out=maxima)
            argmaxima[maxima == prob] = i

        return argmaxima

    def predict_proba(self, X):
        y = np.zeros((X.shape[0], len(self.results_)))
        for i, (e, init_score) in enumerate(self.results_):
            margins = e.predict(X, raw_score=True)
            y[:, i] = special.expit(margins + init_score)
        y /= np.sum(y, axis=1)[:, np.newaxis]
        return y

In [ ]:
%%time

from sklearn.model_selection import KFold, StratifiedKFold # k-фолдная валидация
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm import Dataset, train
import time
import lightgbm
import warnings
warnings.filterwarnings('ignore')

import gc 

random_state = 42
n_splits = 5 # kfolds
print('num folds', n_splits)
clfs = []
targets = 'label'

kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)


split = None

if split is not None:
    print(1)
    X = df[features][:split]
    y = df[targets][:split]
else:
    X = df[features]
    y = df[targets]
    
#print(X.shape)
#print(y.nunique())

n_estimators = [20 for _ in range(n_splits)]

for i, (train_index, test_index) in enumerate(kf.split(X)):
    
    s = time.time()
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print(X_train.shape)

    train_dataset = Dataset(data = X_train, label = y_train, categorical_feature=cat_features)
    eval_dataset = Dataset(data = X_test, label = y_test, categorical_feature=cat_features)
    
    loss = FocalLoss(alpha=0.75, gamma=2.0)
    #model = OneVsRestLightGBMWithCustomizedLoss(loss=loss)
    
    lgb_params = {
         'learning_rate' : 0.1*0.9, 'n_estimators' : n_estimators[i] +10, 
        
        'objective': 'multiclass', 'num_class' : df['label'].nunique(),
        'uniform_drop' : True,  'boosting': 'gbdt',   # goss 
        'lambda_l2' : 1/100, 'feature_fraction': 0.65, # 0.55
        'bagging_freq': 50, 'min_split_gain': 1/1000,
        'max_bin' : 260,
        'random_seed' : 42, 'drop_seed' : 7575,
        'verbose': -1,
        'nthreads':1
    }

    
    fit_params = {'eval_set': [(X_test, y_test)],
                  'num_boost_round': 200,
                  #'early_stopping_rounds': 30,
                   
                  'verbosity': -1,
                  #'verbose_eval': 10,
                  'nthreads': 1,
                 }

    loss = FocalLoss(alpha=0.75, gamma=2.0)
    model = OneVsRestLightGBMWithCustomizedLoss(loss=loss, n_jobs=10)

    model.fit(X_train, y_train, **fit_params)


    clfs.append(model)
    f = time.time()
    print(i,'split done')
    print('final time in hours: ', (f - s)/3600)


In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold # k-фолдная валидация
from lightgbm import LGBMClassifier ,LGBMRegressor
from lightgbm import Dataset, train
import optuna.integration.lightgbm as optuna_lgbm

import optuna

import gc 
import warnings
warnings.filterwarnings('ignore')

import gc 

random_state = 42
n_splits = 10 # kfolds
print('num folds', n_splits)
regressors = []
targets = 'time_to_under'

n_estimators = [100, 300, 400, 300, 400, 300, 300, 300, 400, 500] # best!!

kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

X = df[features]#[:30000]#.drop(targets, axis=1, errors='ignore')#.copy()
y = df[targets]#[:30000]


X = X[y.notna()]
y = y[y.notna()].astype(float)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    train_dataset = Dataset(data = X_train, label = y_train, categorical_feature=cat_features)
    eval_dataset = Dataset(data = X_test, label = y_test, categorical_feature=cat_features)

    print(f'fold {i}', X_train.shape, y_train.shape)

    lgb_params = {
        # 'learning_rate' : 0.1*0.9,
        'n_estimators' : n_estimators[i] + 1000, 
        'objective': 'l2', #'num_class' : df['label'].nunique(), 
        'metric': 'l2',
        'nthreads': 1,

       # 'uniform_drop' : True, 
        'boosting': 'gbdt',   # goss 
        'lambda_l2' : 3/10000, 'feature_fraction': 0.97, # 0.55
        'bagging_fraction': 0.8, 'min_child_samples': 56,
        'bagging_freq': 1,

        'random_seed' : 42, 'drop_seed' : 7575,
        'verbose': -1,
        'lambda_l1': 0.51,
        'num_leaves': 517,
    }
    


    r = lgbm.train(params = lgb_params, train_set = train_dataset,
                   verbose_eval = 100, valid_sets=eval_dataset,
                   callbacks=[lightgbm.early_stopping(stopping_rounds=50,verbose=0)]
                           )
    regressors.append(r)


In [ ]:
test = pd.read_csv('test_dataset_test (1).csv')
test, _ = handle(test)
test, ticket_encoder = encode(test, 'ticket_type_nm',ticket_encoder)
test, station_encoder = encode(test, 'station_id',station_encoder)
test, line_encoder = encode(test, 'line_id',line_encoder)
test, entrance_encoder = encode(test, 'entrance_id',entrance_encoder)

print(test.shape)

test.head()

In [ ]:

from collections import Counter
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

label_decoder = {k:v for v, k in label_encoder.items()}

labels = []
times = []

for chunk in tqdm(np.array_split(test[features], 1000)):
    
    
    labels_drop = []
    for n, clf in enumerate(clfs):
        chunk[f'label_{n}'] = clf.predict(chunk[features])
        labels_drop.append(f'label_{n}')
        
    labels.extend(chunk.apply(
        lambda x: Counter([label_decoder[x[f]] for f in labels_drop]).most_common()[0][0], axis=1).values)
    
    regr_drop = []
    for n, reg in enumerate(regressors):
        chunk[f'ttu_{n}'] = reg.predict(chunk[features])
        regr_drop.append(f'ttu_{n}')
        
    times.extend(chunk.apply(
        lambda x: np.mean([x[f] for f in regr_drop]), axis=1).values)
    

    #labels.extend(model1.predict(chunk[features]).tolist())
   # times.extend(model2.predict(chunk[features]).tolist())

test['label'] = labels
test['time_to_under'] = times



In [ ]:
test[['id','time_to_under','label']].to_csv('augmentation.csv', index=None, sep=',')
test['label'].value_counts()